# Big query sql

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from getpass import getpass
pjt_code = getpass('ProjectCode: ')

ProjectCode: ··········


In [ ]:
%%bigquery --project {pjt_code} df
DECLARE start_date DATE DEFAULT "2019-06-01";

WITH 
-- get all talk data with filling the null thread_ts as ts
all_talks_with_filled_thread_ts AS (
  SELECT
    channel_id, ts AS thread_ts, ts, talk_user, talk_id
  FROM
    mgm.talk
  WHERE
    target_date > start_date AND
    thread_ts IS NULL AND -- fill null thread_ts as ts
    talk_user IS NOT NULL -- exclude bot talks
  UNION ALL
  SELECT
    channel_id, thread_ts, ts, talk_user, talk_id
  FROM
    mgm.talk
  WHERE
    target_date > start_date AND
    thread_ts IS NOT NULL AND -- if thread_ts is not null no replace
    talk_user IS NOT NULL -- exclude bot talks
),
-- get first talk ts in all threads
-- mainly first talk ts = thread_ts but in some case not same
first_talk_in_threads AS (
  SELECT
    channel_id, thread_ts, MIN(ts) AS ts -- get MIN(ts) because some first talk ts > thread_ts
  FROM
    all_talks_with_filled_thread_ts
  GROUP BY
    channel_id, thread_ts
  ORDER BY thread_ts
),
-- join the talk_user, talk_id for first talks
thread_first_user_master AS (
  SELECT
    f.channel_id, f.thread_ts, f.ts, a.talk_user, a.talk_id
  FROM
    first_talk_in_threads f
      LEFT JOIN
        all_talks_with_filled_thread_ts a
      ON
        f.channel_id = a.channel_id AND f.thread_ts = a.thread_ts AND f.ts = a.ts
),
-- get the reply talks (non first talks) on each threads
reply_talk_data AS (
  SELECT
    DISTINCT channel_id, thread_ts, ts, talk_user, talk_id, target_date
  FROM
    mgm.talk AS raw
  WHERE
    target_date > start_date AND talk_user IS NOT NULL AND
    NOT EXISTS(
      SELECT talk_id
      FROM thread_first_user_master AS mstr
      WHERE raw.talk_id = mstr.talk_id
    )
  ORDER BY thread_ts
)

SELECT
  rp.channel_id,
  ROW_NUMBER() OVER(PARTITION BY rp.channel_id, rp.thread_ts ORDER BY rp.thread_ts, rp.ts, rp.channel_id) AS reply_num,
  rp.ts AS reply_ts, rp.talk_id AS reply_id, rp.talk_user AS reply_user,
  tm.thread_ts, tm.talk_id AS talk_id, tm.talk_user AS talk_user, rp.target_date
FROM 
  reply_talk_data rp
    LEFT JOIN
      thread_first_user_master tm
    ON
      rp.channel_id = tm.channel_id AND rp.thread_ts = tm.thread_ts
ORDER BY
  rp.thread_ts, rp.ts, rp.channel_id
;

In [ ]:
df

,channel_id,reply_num,reply_ts,reply_id,reply_user,thread_ts,talk_id,talk_user,target_date
0,CJP6483K2,1,2019-06-23 05:58:36.000300+00:00,d796bfa5-b7a6-4279-baae-0e7ac61fee01,UJRAL005U,2019-06-22 15:07:58.000300+00:00,d9194be0-d223-49c3-a081-84f24965c391,UKFNDDER1,2019-06-24
1,CJPPRCKBN,1,2019-06-23 05:58:36.000300+00:00,359d44e7-f592-4083-860c-60a0a533fba0,UJRAL005U,2019-06-22 15:07:58.000300+00:00,f32ebdda-c165-4add-b88a-cb3aaeaf49de,UKFNDDER1,2019-06-24
2,CJQ8QNGBF,1,2019-06-23 05:58:36.000300+00:00,2ca04290-003b-4308-aeb8-e03115042a3d,UJRAL005U,2019-06-22 15:07:58.000300+00:00,8dfc2ebc-48f5-4e73-af9d-9a7d6db17f67,UKFNDDER1,2019-06-24
3,CJP6483K2,2,2019-06-23 06:01:44.002000+00:00,20001c5b-fa6c-4034-9d46-ed59a3b13300,UKFNDDER1,2019-06-22 15:07:58.000300+00:00,d9194be0-d223-49c3-a081-84f24965c391,UKFNDDER1,2019-06-24
4,CJPPRCKBN,2,2019-06-23 06:01:44.002000+00:00,2bc1b18e-97b7-471b-8dfc-abfb7312bd2f,UKFNDDER1,2019-06-22 15:07:58.000300+00:00,f32ebdda-c165-4add-b88a-cb3aaeaf49de,UKFNDDER1,2019-06-24
...,...,...,...,...,...,...,...,...,...
18351,CJCNV9LG2,10,2020-07-16 14:51:18.239900+00:00,07801a08-aa4f-4c6e-b70d-1192a0720dfc,UJRAL005U,2020-07-16 11:35:05.235400+00:00,3fd7227c-6d9b-408d-a1b6-207f9ce86a8c,URG3NEHGW,2020-07-17
18352,CJCNV9LG2,11,2020-07-16 14:54:22.240100+00:00,e8c128e3-32ff-4d2d-8af8-539b96ff698d,U010Y77JA2W,2020-07-16 11:35:05.235400+00:00,3fd7227c-6d9b-408d-a1b6-207f9ce86a8c,URG3NEHGW,2020-07-17
18353,CJP6483K2,1,2020-07-16 13:46:03.124400+00:00,cc1ce71c-ed90-447c-841b-b537fc5d6dc5,UJRAL005U,2020-07-16 13:17:28.116200+00:00,a3a8a472-a52a-4773-8528-5a2362385b4b,U016YJ3TNE7,2020-07-17
18354,CJP6483K2,2,2020-07-16 13:51:01.124700+00:00,c64fb1a3-fb3c-4f37-a0fe-05eaf20b2676,U016YJ3TNE7,2020-07-16 13:17:28.116200+00:00,a3a8a472-a52a-4773-8528-5a2362385b4b,U016YJ3TNE7,2020-07-17


In [ ]:
import pandas as pd
df.to_csv('reply_table.csv', index=False)

In [ ]:
%%bigquery --project {pjt_code} df_user_master
DECLARE start_date DATE DEFAULT "2019-06-01";

-- 各ユーザIDごとの最新日を取得
WITH user_max_trg_dates AS (
  SELECT
    user_id, MAX(target_date) AS max_target_date
  FROM
    mgm.user
  WHERE target_date > start_date
  GROUP BY user_id
),
-- 各ユーザIDごとの発言数
user_talk_counts AS (
  SELECT
    talk_user, COUNT(DISTINCT talk_id) AS talk_count
  FROM
    mgm.talk
  WHERE target_date > start_date
  GROUP BY talk_user
),
-- 最新日とトーク数をマージ
user_master_data AS(
  SELECT
    m.user_id, m.max_target_date, c.talk_count
    FROM user_max_trg_dates m
      JOIN user_talk_counts c
      ON m.user_id = c.talk_user
),

-- userテーブルに一部日付において重複があるので、それを消去（JOIN時に邪魔）
all_distinct_table AS(
  SELECT
    DISTINCT user_id, real_name_normalized, display_name_normalized, target_date
  FROM
    mgm.user
  WHERE
    target_date > start_date
),
-- 各target_date毎のuser_idとname
-- display_name=nullの方もいるので、その場合はreal_nameを使用
name_normalized_table AS(
  SELECT
    user_id, real_name_normalized AS name, target_date
  FROM
    all_distinct_table
  WHERE
    display_name_normalized IS NULL
  UNION ALL
  SELECT
    user_id, display_name_normalized AS name, target_date
  FROM
    all_distinct_table
  WHERE
    display_name_normalized IS NOT NULL
  ORDER BY target_date, user_id
)
-- 各ユーザの最新日にて該当日のnameをJOIN
SELECT
  m.user_id, u.name, u.target_date, m.talk_count
FROM user_master_data m
  INNER JOIN
    name_normalized_table u
  ON
    m.user_id = u.user_id AND m.max_target_date = u.target_date
ORDER BY m.max_target_date, m.user_id
;


In [ ]:
df_value_count = df_user_master['name'].value_counts().to_frame()
double_name = df_value_count[df_value_count['name']>1].index.tolist()

In [ ]:
for dn in double_name:
  df_tmp = df_user_master[df_user_master['name']==dn]

In [ ]:
df_user_master.to_csv('user_master.csv', index=False)

In [ ]:
df_user_master.head()

,user_id,name,target_date,talk_count,count
0,UKFNDDER1,こんちゃん,2020-04-03,108,1
1,UKLQEKEV7,本間 翔太,2020-04-03,22,1
2,UKP483XHR,u950527,2020-04-03,9,1
3,UKP4HTQ1G,Naoki,2020-04-03,23,1
4,UKPCQAQLA,鈴木淳平,2020-04-03,29,1


In [ ]:
df_user_master['count'] = 1
df_user_master.groupby(['user_id', 'max_target_date']).count().max()

KeyError: ignored

In [ ]:
df_user_master

,user_id,max_target_date,count
0,UKG6BDAPL,2020-06-16,1
1,UM9G63GUR,2020-05-11,1
2,UPAD4C0B0,2020-04-03,1
3,UTKK6HK2B,2020-07-12,1
4,U012PJ9KJNS,2020-07-12,1
...,...,...,...
405,URDDX224S,2020-07-12,1
406,URPSGF5SP,2020-07-12,1
407,UR126F540,2020-06-09,1
408,URDDG8EVA,2020-07-12,1


In [ ]:
%%bigquery --project {pjt_code} df_test
DECLARE start_date DATE DEFAULT "2019-06-01";
WITH distinct_table AS(
  SELECT
    DISTINCT user_id, real_name_normalized, display_name_normalized, target_date
  FROM
    mgm.user
  WHERE
    target_date > start_date
)
SELECT
  user_id, real_name_normalized AS name, target_date
FROM
  distinct_table
WHERE
  display_name_normalized IS NULL
UNION ALL
SELECT
  user_id, display_name_normalized AS name, target_date
FROM
  distinct_table
WHERE
  display_name_normalized IS NOT NULL
ORDER BY target_date, user_id

In [ ]:
df_test

,user_id,name,target_date
0,UJKFAPBCJ,岡村龍弥,2019-06-24
1,UJRAL005U,村上智之 ｷﾞﾙﾄﾞ代表 筋ﾄﾚで学ぶﾃﾞｰﾀ分析,2019-06-24
2,UJRDMQSAD,勝又健太 kenta.katsumata,2019-06-24
3,UKEMYD9R8,tetsuroito,2019-06-24
4,UKFLR3VHR,相川 仁,2019-06-24
...,...,...,...
86513,UUAUDBR1N,戸嶋 龍哉,2020-07-12
86514,UUMB12SGZ,永田ゆかり Yukari Nagata,2020-07-12
86515,UV1GRG3UZ,Kazuhii,2020-07-12
86516,UV1J8NN6B,funakoshi,2020-07-12


In [ ]:
df_test.groupby(['user_id', 'target_date']).count().max()

real_name_normalized       1
display_name_normalized    1
dtype: int64

In [ ]:
%%bigquery --project {pjt_code} df_test
DECLARE start_date DATE DEFAULT "2019-06-01";
SELECT DISTINCT *
FROM mgm.user
WHERE target_date > start_date


In [ ]:
df_test

,user_id,real_name_normalized,display_name_normalized,target_date
0,URDT39KGV,後藤,None,2020-05-03
1,U012JJPCQP5,yoshizirou,None,2020-05-03
2,U012WSURE92,RAI,RAI,2020-05-03
3,UV1GRG3UZ,Kazuhii,Kazuhii,2020-05-03
4,UTK77QX52,tobi,SHIMADA,2020-05-03
...,...,...,...,...
86513,UNLNYN8VA,杉本 光一,None,2019-10-13
86514,UMFDK5ETW,Y,Y,2019-10-13
86515,ULM6FBU11,tomo.h,tomo.h,2019-10-13
86516,UMJ5SQSCF,Keisuke Sugiyama,Keisuke Sugiyama,2019-10-13


In [ ]:
%%bigquery --project {pjt_code} df_test
DECLARE start_date DATE DEFAULT "2019-06-01";

-- 各ユーザIDごとの最新日を取得
WITH user_max_trg_dates AS (
  SELECT
    user_id, MAX(target_date) AS max_target_date
  FROM
    mgm.user
  WHERE target_date > start_date
  GROUP BY user_id
),
-- 各ユーザIDごとの発言数
user_talk_counts AS (
  SELECT
    talk_user, COUNT(DISTINCT talk_id) AS talk_count
  FROM
    mgm.talk
  WHERE target_date > start_date
  GROUP BY talk_user
)
-- 最新日とトーク数をマージ
  SELECT
    m.user_id, m.max_target_date, c.talk_count
    FROM user_max_trg_dates m
      JOIN user_talk_counts c
      ON m.user_id = c.talk_user

In [ ]:
df_test

,user_id,max_target_date,talk_count
0,UML7CC4N8,2020-04-03,9
1,UMRST297C,2020-07-13,5
2,UN13XU726,2020-04-03,1
3,UM6RV9SP4,2020-04-03,21
4,UTRSFL2LS,2020-07-13,53
...,...,...,...
360,URDDX224S,2020-07-13,721
361,URPSGF5SP,2020-07-13,30
362,UR126F540,2020-06-09,96
363,URDDG8EVA,2020-07-13,116
